In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
plt.style.use('seaborn')

In [3]:
'''
- check tensorflow and keras version
'''
print(f'Tensorflow GPU Version: {tf.__version__}')
print(f'Eager Execution is: {tf.executing_eagerly()}')
print(f'Keras Version: {tf.keras.__version__}')

Tensorflow GPU Version: 2.0.0
Eager Execution is: True
Keras Version: 2.2.4-tf


In [4]:
'''
- check GPU 
'''
var = tf.Variable([3,3])
if tf.test.is_gpu_available():
    print('Running on GPU')
else:
    print('Runing on CPU')

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Running on GPU


Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

# MLP model

In [10]:
'''
- MLP layer
'''
class MLP(tf.keras.layers.Layer):
    def __init__(self, units=3, activation=None, trainable=True, name=None, dtype=tf.float32, **kwargs):
        super(MLP, self).__init__( name=name, trainable = trainable, dtype=dtype, **kwargs)
        self.units = units
        self.__activation_name = activation
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),initializer='random_normal',)
        self.b = self.add_weight(shape=(self.units,),initializer='random_normal')
        
    def get_config(self):
        config_dic ={
            'units':self.units,
            'activation': self.__activation_name, 
            'trainable_weights & bias':self.trainable_weights,
            'non-trainable_weights & bias':self.non_trainable_weights,
        }
        config = super(MLP,self).get_config()
        config.update(config_dic)
        return config
    
    def call(self, inputs, training = None):
        if tf.rank(inputs)==1:
            inputs = tf.keras.backend.expand_dims(inputs, axis=0)
        linear_combination = tf.matmul(inputs, self.w)+self.b 
        return self.activation(linear_combination)

# Multi-layer Perceptron Model

In [7]:
'''
- subclass model
'''
class MLP_Model(tf.keras.Model):
    def __init__(self, **kwargs):
        super(MLP_Model, self).__init__(self, **kwargs)
        self.mlp1 = MLP(256, activation='sigmoid')
        self.mlp2 = MLP(128, activation='sigmoid')
        self.mlp3 = MLP(10, acitivation='softmax')
        
    def call(self, inputs):
        opt = self.mlp1(inputs)
        opt = self.mlp2(opt)
        opt = self.mlp3(opt)
        return opt

# Logistic Regression Model

In [12]:
'''
- logistic regression
- perform binary classification with multi-labels
'''
class LogisticRegression(tf.keras.Model):
    def __init__(self,labels=1, **kwargs):
        super(LogisticRegression, self).__init__(self, **kwargs)
        self.label = labels
        self.lglayer = MLP(labels, activation='sigmoid')
    
    def call(self, inputs):
        opt= self.lglayer(inputs)
        return opt

In [13]:
lg_model = LogisticRegression(labels=3)

# KNN

In [17]:
a = tf.Variable([[40, 30, 20, 10], [10, 20, 30, 40]],dtype=tf.float32)
k_value, k_index = tf.math.top_k(a,2)

In [18]:
k_value

<tf.Tensor: id=40, shape=(2, 2), dtype=float32, numpy=
array([[40., 30.],
       [40., 30.]], dtype=float32)>

In [19]:
k_index

<tf.Tensor: id=41, shape=(2, 2), dtype=int32, numpy=
array([[0, 1],
       [3, 2]], dtype=int32)>